In [1]:
import os
import glob
import sys
import numpy as np
import pandas as pd
import hotspot
import matplotlib.pyplot as plt
import matplotlib.colors
import seaborn as sns
import pickle
import loompy as lp
sys.setrecursionlimit(3000)

### input files

In [2]:
counts_file = "HCC04_T_bin50_expmtx_nvg5000.loom"
pos_file = "HCC04_T_bin50_cellpos.csv"
outdir = "HCC04_T_bin50"
proname = 'HCC04_T_bin50_expmtx_nvg5000'
if not os.path.exists(outdir):
    os.makedirs(outdir)

### load expression counts and spatial positions

In [3]:
FTYPE = os.path.splitext(os.path.basename(counts_file))[1]
SUPPORTED_SC_EXP_FTYPE = [".csv", ".loom"]
if not FTYPE in SUPPORTED_SC_EXP_FTYPE:
    raise SystemExit(
        f'Supported expression matrix formats are {SUPPORTED_SC_EXP_FTYPE}, filename extension must be consistent with contents')
HS_RESULTS = ''.join([outdir,"/",proname,"_hs_results.p"])
LCZ = ''.join([outdir, "/", proname, "_lcz.p"])
MODULES = ''.join([outdir, "/", proname, "_modules.p"])
HOTSPOT = ''.join([outdir, "/", proname, "_hotspot.p"])

if FTYPE==".csv":
    counts = pd.read_csv(counts_file, index_col=0) # Takes a while, ~10min
if FTYPE==".loom":
    lf = lp.connect(counts_file, mode='r', validate=False)
    counts = pd.DataFrame(lf[:, :],
                             index=lf.ra["Gene"],
                             columns=lf.ca["CellID"],
                             dtype=np.int64)
    lf.close()

pos = pd.read_csv(pos_file, index_col=0)

### align indices

In [4]:
pos = pos.loc[counts.columns, :]
counts = counts.loc[:, pos.index]
barcodes = pos.index.values
pos = pd.DataFrame(
    {
        'X': pos.X,
        'Y': pos.Y,
    }, index=pos.index
)

num_umi = counts.sum(axis=0)

### create the Hotspot object and the neighborhood graph

In [5]:
hs = hotspot.Hotspot(counts, model='normal', latent=pos, umi_counts=num_umi)
hs.create_knn_graph(weighted_graph=False, n_neighbors=30)
hs_results = hs.compute_autocorrelations(jobs=20)
with open(HS_RESULTS, "wb") as f:
    pickle.dump(hs_results,f)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5000/5000 [02:22<00:00, 35.08it/s]


### select genes with significant spatial autocorrelation

In [6]:
hs_genes = hs_results.index[hs_results.FDR < 0.05]

### compute pair-wise local correlations between selected genes

In [7]:
lcz = hs.compute_local_correlations(hs_genes, jobs=20)

with open(LCZ, "wb") as f:
    pickle.dump(lcz,f)


modules = hs.create_modules(
    min_gene_threshold=20, core_only=False, fdr_threshold=0.05
)

with open(MODULES, "wb") as f:
    pickle.dump(modules, f)


with open(HOTSPOT, "wb") as f:
    pickle.dump(hs,f)

plt.rcParams['figure.figsize'] = (15.0, 12.0)

hs.plot_local_correlations()

plt.savefig(''.join([outdir, "/", proname, "_module_number.png"]), dpi = 600)
plt.close()

results = hs.results.join(hs.modules)
results.to_csv(''.join([outdir, "/", proname, "_Cluster.csv"]))

module_scores = hs.calculate_module_scores()
module_scores.to_csv(''.join([outdir, "/", proname, "_ModuleScore.csv"]))

if not os.path.exists(f'{outdir}/ModuleFig'):
    os.makedirs(f'{outdir}/ModuleFig')
for module in range(1, hs.modules.max()+1):
    scores = hs.module_scores[module]

    vmin = np.percentile(scores, 1)
    vmax = np.percentile(scores, 99)

    plt.scatter(x=hs.latent.iloc[:, 0],
                y=hs.latent.iloc[:, 1],
                s=8,
                c=scores,
                vmin=vmin,
                vmax=vmax,
                edgecolors='none'
                )
    axes = plt.gca()
    for sp in axes.spines.values():
        sp.set_visible(False)
    plt.xticks([])
    plt.yticks([])
    plt.title('Module {}'.format(module))
    plt.savefig(f'{outdir}/ModuleFig/Module{module}.png')
    plt.close()

Computing pair-wise local correlation on 1255 features...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 786885/786885 [05:11<00:00, 2527.61it/s]


Computing scores for 13 modules...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:20<00:00,  1.55s/it]
